In [19]:
import os
import pandas as pd
import re
import numpy as np
from datetime import datetime
from openpyxl import load_workbook
import warnings


warnings.filterwarnings("ignore", category=UserWarning, module='openpyxl')


# input

your_ing_id = 'XO21BM'
date = datetime.strptime('2024', "%Y")

directory_path0 = os.getcwd()
directory_path = path = f"/Users/{your_ing_id}/Library/CloudStorage/OneDrive-SharedLibraries-ING/Product Evaluation and Risk Assessment Library (PEARL) - PEARL_Repository"
directory_path2 = path = f"/Users/{your_ing_id}/Library/CloudStorage/OneDrive-SharedLibraries-ING/Product Evaluation and Risk Assessment Library (PEARL) - MI Dashboard"        
directory_path3 = directory_path0 + '/Intermediate results/'            

In [20]:
# folder and ID mapping
overview = pd.read_csv('PEARL List (1).csv', sep=',', low_memory=False)
overview['Folder_URL_txt'] = overview['Folder_URL_txt'].str.replace('#$@', ' ')
overview['Folder'] = overview['Folder_URL_txt'].str.replace(r'https://ing.sharepoint.com/sites/PEARL_cs/Shared Documents/Pearl_Repository/', '')
overview['Folder'] = overview['Folder'].str.replace(r'https://ing.sharepoint.com/sites/PEARL_cs/Shared Documents/PEARL_Repository/', '')
cols_overview = ['Folder', 'ID']
overview = overview[cols_overview]
overview['ID'] = overview['ID'].astype(str)

In [21]:
# reading MI dashboard
file_name = 'MI Dashboard.xlsm'
file_path = os.path.join(directory_path2, file_name)

In [22]:
# Changes
# reading MI dashboard
file_name = 'MI Dashboard.xlsm'
file_path = os.path.join(directory_path2, file_name)

# Changes
mi_dash_changes = pd.read_excel(file_path, sheet_name="Management Info Changes", engine="openpyxl", skiprows=2)
mi_dash_changes.rename(columns={'PEARL ID': 'ID'}, inplace=True)

mi_dash_changes.loc[mi_dash_changes['End Date'] == '(blank)', 'End Date'] = np.nan
mi_dash_changes.loc[mi_dash_changes['Start Date'] == '(blank)', 'Start Date'] = np.nan
mi_dash_changes['End Date'] = pd.to_datetime(mi_dash_changes['End Date'])
mi_dash_changes['Start Date'] = pd.to_datetime(mi_dash_changes['Start Date'])


mi_dash_changes['Type'] = 'Change'

In [23]:

# Reviews
mi_dash_review = pd.read_excel(file_path, sheet_name="Management Info Reviews", engine="openpyxl", skiprows=2)
mi_dash_review.rename(columns={'RJT Review PEARL ID': 'ID', 
                               'RJT Review Start Date': 'Start Date', 
                               'RJT Review Approval Date': 'End Date', 
                               'RJT Review Status': 'Status'}, inplace=True)

# filtering only the relevant ones for our current analysis on the complete ones
mi_dash_review.loc[mi_dash_review['End Date'] == '(blank)', 'End Date'] = np.nan
mi_dash_review.loc[mi_dash_review['Start Date'] == '(blank)', 'Start Date'] = np.nan
mi_dash_review['End Date'] = pd.to_datetime(mi_dash_review['End Date'])
mi_dash_review['Start Date'] = pd.to_datetime(mi_dash_review['Start Date'])

mi_dash_review['Type'] = 'Review'

In [24]:
#mi_risk_module = pd.read_excel(file_path, sheet_name="Risk Modules", engine="openpyxl", skiprows=2)

# creating overview so we know in which risk assesments we are interested in
final_overview = pd.concat([mi_dash_changes, mi_dash_review])


cond = ~(final_overview['End Date'] < date) & ~(final_overview['ID'].isna())
final_overview = final_overview[cond]
cols = ['ID', 'Start Date', 'End Date', 'Process Category', 'Tribe', 'Status', 'Type', 'Duration']
final_overview = final_overview[cols]
final_overview = final_overview.merge(overview, how = 'left', on = 'ID')


final_overview['End Year'] = final_overview['End Date'].dt.year

cond = final_overview['Process Category'].isna()
final_overview.loc[cond, 'Process Category'] = 'empty'

final_overview_grouped = final_overview.groupby([
    'Process Category', 
    'End Year', 
    'Type']).agg({'ID': 'count'}).reset_index()

final_overview.to_csv('final_overview.csv', sep=';', index = False)

final_overview = pd.read_csv('final_overview.csv', sep=';')

# prep for the loop
# result1 = pd.DataFrame()
# result2 = pd.DataFrame()
# result3 = pd.DataFrame()
# result4 = pd.DataFrame()

In [25]:
def find_matching_strings(strings):
    matching_strings = []
    for string in strings:
        if string.lower().startswith('risk') and 'journey' in string.lower() and string.lower().endswith('xlsm'):
            matching_strings.append(string)
    
    return matching_strings

folders_to_check = final_overview.loc[:, 'Folder']

# for testing and cheking specific folder ########
item_to_find = r'Hypotheken - WUB/WUB Hypotheek met Beleggingsrekening/2024-04-15 Intermediate Review product'
temp_index = 0
for f in folders_to_check:
    
    if f == item_to_find:
        break
    
    temp_index = temp_index + 1
    
folder = folders_to_check.iloc[temp_index] # for testing
folder = folders_to_check.iloc[859] # for testing

In [32]:
complete_folder_path

'/Users/XO21BM/Library/CloudStorage/OneDrive-SharedLibraries-ING/Product Evaluation and Risk Assessment Library (PEARL) - PEARL_Repository/Credit Cards/Creditcard/2024-11-14 Change authentication level cards IRIS  Risk assessment MIA ISSUE10060964/'